# Szemantikus Kernel

Ebben a kódmintában a [Szemantikus Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI keretrendszert fogod használni egy alapvető ügynök létrehozásához.

Ennek a mintának a célja, hogy bemutassa azokat a lépéseket, amelyeket később a további kódmintákban fogunk alkalmazni a különböző ügynöki minták megvalósításakor.


## Importálja a szükséges Python csomagokat


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Az ügyfél létrehozása

Ebben a példában a [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) szolgáltatást fogjuk használni az LLM eléréséhez.

Az `ai_model_id` értéke `gpt-4o-mini`. Próbálja meg megváltoztatni a modellt egy másikra, amely elérhető a GitHub Models piacterén, hogy lássa a különböző eredményeket.

Ahhoz, hogy használni tudjuk az `Azure Inference SDK`-t, amelyet a GitHub Models `base_url`-jához használunk, a Semantic Kernel-en belül az `OpenAIChatCompletion` csatlakozót fogjuk alkalmazni. További [elérhető csatlakozók](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) is rendelkezésre állnak, amelyek lehetővé teszik a Semantic Kernel használatát más modell szolgáltatók esetében.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Az ügynök létrehozása

Az alábbiakban létrehozzuk a `TravelAgent` nevű ügynököt.

Ebben a példában nagyon egyszerű utasításokat használunk. Az utasításokat módosíthatod, hogy megnézd, hogyan reagál az ügynök eltérően.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Az ügynök futtatása

Most futtathatjuk az ügynököt egy `ChatHistoryAgentThread` típusú szál definiálásával. Az ügynök invoke_stream `messages` kulcsszó argumentumában megadjuk a szükséges rendszerüzeneteket.

Miután ezek definiálva lettek, létrehozunk egy `user_inputs`-ot, amelyet a felhasználó küld az ügynöknek. Ebben az esetben ezt az üzenetet állítottuk be: `Plan me a sunny vacation`.

Nyugodtan változtasd meg ezt az üzenetet, hogy lásd, hogyan reagál az ügynök különböző módon.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Fontos információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
